<a href="https://colab.research.google.com/github/jublipatra99/portfolio/blob/master/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10 # use Keras built-in CIFAR10 dataset
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D 

In [8]:
 #set parameters
batch_size = 32
num_classes = 10

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
# Prepare train & test Data
y_train = to_categorical(y_train, num_classes) # one-hot encoding
y_test = to_categorical(y_test, num_classes)   # one-hot encoding

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0 # convert to floating point
x_test /= 255.0  # convert to floating point

In [11]:
# Build Model
model = keras.models.Sequential()
# 1st Conv block
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 2nd Conv block
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# Fully-Connected layer
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0

In [13]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


In [14]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

In [15]:
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [16]:
# Train Model 
num_epochs=10
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=num_epochs, validation_data=(x_test, y_test))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
1563/1563 [==============================] - 200s 128ms/step - loss: 1.6121 - accuracy: 0.4043 - val_loss: 1.2682 - val_accuracy: 0.5481
Epoch 2/10
1563/1563 [==============================] - 201s 128ms/step - loss: 1.2922 - accuracy: 0.5345 - val_loss: 1.0697 - val_accuracy: 0.6158
Epoch 3/10
1563/1563 [==============================] - 200s 128ms/step - loss: 1.1543 - accuracy: 0.5898 - val_loss: 0.9936 - val_accuracy: 0.6512
Epoch 4/10
1563/1563 [==============================] - 203s 130ms/step - loss: 1.0639 - accuracy: 0.6227 - val_loss: 0.8584 - val_accuracy: 0.6986
Epoch 5/10
1563/1563 [==============================] - 204s 131ms/step - loss: 1.0117 - accuracy: 0.6417 - val_loss: 0.8271 - val_accuracy: 0.7086
Epoch 6/10
1563/1563 [==============================] - 203s 130ms/step - loss: 0.9691 - accuracy: 0.6620 - val_loss: 0.8478 - val_accuracy: 0.7005
Epoch 7/10
1563/1563 [==============

In [17]:
# Evaluate Model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 9s 27ms/step - loss: 0.7258 - accuracy: 0.7503
Test loss: 0.7258016467094421
Test accuracy: 0.7502999901771545


In [18]:
from keras.models import load_model 
model.save('project_model.h5')

In [19]:
import pandas as pd
import numpy as np
test_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/cifar_image_flattened_pixels.csv")

In [20]:
test_data1=np.array(test_data)
test_data1=test_data1.reshape(2000,32,32,3)

In [21]:
predictions=model.predict(test_data1)

In [22]:
res=pd.DataFrame(predictions)
res.index=test_data.index

In [24]:
res=np.argmax(predictions,axis=1)

In [25]:
from google.colab import files

In [35]:
for i in res:
  print(i)

0
8
7
9
5
1
7
7
1
5
1
1
1
1
5
7
0
0
7
1
5
9
7
1
9
1
1
7
0
7
7
1
7
9
2
0
1
1
1
7
7
1
5
1
7
5
1
2
1
1
0
1
7
5
8
1
1
1
1
1
7
1
7
6
1
1
9
1
7
1
1
1
9
1
0
2
7
1
7
8
7
5
0
9
1
7
5
8
1
1
1
1
9
0
1
0
0
7
1
7
1
1
7
7
1
1
1
1
7
0
0
2
1
0
1
1
1
5
0
7
1
1
1
0
7
2
1
1
1
0
9
5
0
1
2
7
9
4
9
1
7
9
1
7
7
1
5
8
1
9
1
7
0
0
5
5
5
1
7
8
5
1
9
0
7
7
1
1
8
1
2
9
9
0
1
1
0
7
3
7
1
7
2
7
0
1
7
7
9
0
7
1
0
7
0
1
1
7
1
1
1
1
1
6
8
0
1
1
0
3
1
8
1
5
1
8
2
1
1
7
7
1
0
7
1
7
3
0
5
7
5
6
9
1
9
1
0
8
0
7
1
7
0
5
5
7
5
0
1
1
9
8
9
1
1
5
5
0
0
0
0
0
1
1
0
1
0
1
0
2
8
0
8
9
7
1
1
9
7
1
0
7
1
1
0
0
1
1
1
1
1
1
1
1
9
1
1
9
1
1
2
1
1
0
0
1
4
9
5
9
1
1
7
1
1
1
2
5
9
0
7
9
1
9
7
9
9
1
8
8
7
7
1
7
0
9
1
0
5
1
5
0
1
0
2
9
1
5
8
9
0
2
0
1
7
1
1
1
9
1
0
7
1
1
1
7
1
1
9
7
0
1
1
0
5
0
5
9
1
5
3
9
0
0
1
5
5
1
7
0
7
7
7
5
1
1
1
2
7
7
1
1
7
8
7
9
1
1
1
9
1
5
0
7
9
1
1
1
7
1
1
0
1
1
9
0
0
7
9
1
7
1
0
1
0
1
9
4
1
1
1
1
0
9
9
0
1
9
9
5
1
5
1
5
7
1
1
7
1
1
7
7
1
7
7
1
9
7
0
0
1
7
5
1
8
9
1
1
7
7
8
0
7
1
3
9
7
5
1
7
1
0
7
9
5
7
0
1
1
3
